# File/Environment Preparation

In [36]:
import os
print(os.listdir())

# uncomment if mounting google drive
directory_path = ''
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
# directory_path = '/content/drive/MyDrive/Quantum/'
# sys.path.append('/content/drive/MyDrive/Quantum')
#print(os.listdir(directory_path))

['Symmetric_Exclusion_Process _Simulator.ipynb', 'Quantum_Transformer.ipynb', 'dense_small.param', '.DS_Store', 'quantum_experiments', 'DiscreteVariationalParameterizationsDeepV2.py', 'Mutual_Information_Transformer.ipynb', 'DiscreteVariationalParameterizationsDeepV3.py', 'README.md', 'Mutual_Information_Maximizing_Model.ipynb', 'temp.txt', '.ipynb_checkpoints', '.git', 'Quantum_Brickworks_Circuit_Generator.ipynb', 'QuantumSimulatorDataset.py', 'GibbsSampling.py', '.idea']


In [129]:
!pip install torch
!pip install qiskit-aer
!pip install qiskit
!pip install pylatexenc
!pip install tqdm

In [132]:
import torch
import numpy as np
import torch.nn as nn
import DiscreteVariationalParameterizationsDeepV3 as DVP
from torch.autograd.functional import vjp
from torch.autograd.function import Function
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import mutual_info_score
from QuantumSimulatorDataset import QuantumSimulationDatasetFast, generate_circuit_params
from GibbsSampling import BatchedConditionalGibbsSampler, BatchedConditionalDoubleGibbsSampler

# MI Model Definition

In [157]:
class EmbeddingMI3(nn.Module):
    def __init__(self, batch_size, in_dim, out_dim, num_ones):
        super().__init__()
        self.encoder = DVP.BoltzmannBasedEncoder(in_dim=in_dim, out_dim=out_dim)
        self.decoder = DVP.EnergyBasedDecoder(in_dim=out_dim, out_dim=in_dim, num_ones=num_ones)
        self.num_ones = num_ones
        self.embedding_dynamics = DVP.EnergyBasedModelEmbeddingDynamics(dim=out_dim)
        self.loss_func = MutualInformationLossV3.apply
        self.embedding_sampler = BatchedConditionalGibbsSampler(batch_size=batch_size, num_samples=256, # needs to be tuned
                                                                mixing_time=5, # seems like this can be low and still work
                                                                joint_distribution=self.embedding_dynamics)
        self.decoder_sampler = BatchedConditionalDoubleGibbsSampler(batch_size=batch_size, num_samples=256, # needs to be tuned
                                                                mixing_time=24, # seems like this can be low and still work
                                                                joint_distribution=self.decoder, dim=in_dim, num_ones=self.num_ones)
    def test_objective_function(self, x, y):
        w = self.encoder.encoder_sample(x).detach()
        z = self.encoder.encoder_sample(y).detach()
        # print(w, z, x, y)

        w_tilde = self.embedding_sampler.run_batched_gibbs(z).detach()
        x_tilde = self.decoder_sampler.run_batched_gibbs(w).detach()

        return -self.loss_func(self.num_ones, *(z, y, w, x, w_tilde, x_tilde), *self.encoder.params(), *self.decoder.params(), *self.embedding_dynamics.params()), \
                mutual_info_score(w.view(-1), z.view(-1))

# MI Loss and Entropy Functions

In [173]:
class MutualInformationLossV3(Function):
    @staticmethod
    def forward(ctx, *inputs):
        num_ones = inputs[0]
        zywx_w_tilde_ins = inputs[1:7]
        encoder_params = inputs[7:11]
        decoder_params = inputs[11:19]
        embedding_params = inputs[19:27]

        z, y, w, x, _, _ = zywx_w_tilde_ins

        print(encoder_params, decoder_params, embedding_params)
        #print(x[0], '|' ,y[0])
        #print(w[0], '|' ,z[0])

        p_x_w_estimate = DVP.EnergyBasedDecoder.estimated_conditional_log_probability_a_given_b(x, w, num_ones, *decoder_params)
        p_w_x = DVP.BoltzmannBasedEncoder.conditional_log_probability_a_given_b_params(w, x, *encoder_params)
        r_w_z_estimate = DVP.EnergyBasedModelEmbeddingDynamics.estimated_normalized_log_probabilities_w_given_z_params(z, w, *embedding_params)

        out = p_x_w_estimate - p_w_x + r_w_z_estimate
        ctx.num_ones = num_ones
        ctx.save_for_backward(*zywx_w_tilde_ins, *encoder_params, *decoder_params, *embedding_params, r_w_z_estimate, out)
        return out

    @staticmethod
    def backward(ctx, grad_output):
        num_ones = ctx.num_ones
        z, y, w, x, w_tilde, x_tilde = ctx.saved_tensors[0:6]
        encoder_params = ctx.saved_tensors[6:10]
        decoder_params = ctx.saved_tensors[10:18]
        embedding_params = ctx.saved_tensors[18:26]
        r_w_z = ctx.saved_tensors[26]
        MI = ctx.saved_tensors[27]

        # decoder gradient handling
        decoder_unnormalized_probs = lambda x, w, *params: DVP.EnergyBasedDecoder.unnormalized_log_probs_a_given_b_params(num_ones, x, w, *params)
        decoder_expected_unnormalized_probs = lambda x_tilde, w, *params: DVP.EnergyBasedDecoder.expected_unnormalized_log_probs_a_given_b(num_ones, x_tilde, w, *params)

        _, decoder_grad_1 = vjp(decoder_unnormalized_probs, (x, w, *decoder_params), grad_output, create_graph=False)
        _, decoder_grad_2 = vjp(decoder_expected_unnormalized_probs, (x_tilde, w.expand(x_tilde.shape[0], -1, -1), *decoder_params), grad_output, create_graph=False)

        decoder_grad = tuple(map(lambda x, y: x - y, decoder_grad_1[2:], decoder_grad_2[2:]))

        # encoder gradient handling
        _, encoder_grad_term_1 = vjp(DVP.BoltzmannBasedEncoder.conditional_log_probability_a_given_b_params, (w, x, *encoder_params), grad_output * (MI - 1), create_graph=False)
        encoder_grad_term_1 = encoder_grad_term_1[2:]

        _, encoder_grad_term_2 = vjp(DVP.BoltzmannBasedEncoder.conditional_log_probability_a_given_b_params, (z, y, *encoder_params), grad_output * r_w_z, create_graph=False)
        encoder_grad_term_2 = encoder_grad_term_2[2:]

        encoder_grad = tuple(map(lambda x, y: x + y, encoder_grad_term_1, encoder_grad_term_2))

        # embedding gradient handling
        _, embedding_grad_1 = vjp(DVP.EnergyBasedModelEmbeddingDynamics.unnormalized_log_probs_w_given_z_params, (z, w, *embedding_params), grad_output, create_graph=False)
        _, embedding_grad_2 = vjp(DVP.EnergyBasedModelEmbeddingDynamics.expected_unnormalized_log_probs_w_given_z, (z.expand(w_tilde.shape[0], -1, -1), w_tilde, *embedding_params), grad_output, create_graph=False)

        embedding_grad = tuple(map(lambda x, y: x - y, embedding_grad_1[2:], embedding_grad_2[2:]))

        return None, None, None, None, None, None, None, *encoder_grad, *decoder_grad, *embedding_grad

In [174]:
def entropy(X):
    # Flatten the tensor to 1D
    X_flat = X.view(-1)
    
    # Count the occurrences of each unique value
    unique_vals, counts = X_flat.unique(return_counts=True)
    probabilities = counts.float() / counts.sum()
    entropy = -torch.sum(probabilities * torch.log(probabilities))
        
    return entropy.item()


# Dimension Reduction Process

In [175]:
def run_dim_red_process(device, state_space, embedding_space_size, batch_size=256, num_steps=10000):

    model = EmbeddingMI3(batch_size, state_space, embedding_space_size, num_ones=4).to(device)

    # Path to the state dictionary file
    state_dict_path = 'quantum_experiments/initializer.model'
    specific_dict_path = f'quantum_experiments/experiment_{state_space}_{embedding_space_size}_good.model'

    # Check if the state dictionary file exists
    if os.path.exists(directory_path + specific_dict_path):
        state_dict = torch.load(specific_dict_path, map_location=device)
        old_state_dict = model.state_dict()
        old_state_dict['encoder.b'] = state_dict['encoder.b'][:, :embedding_space_size]
        old_state_dict['encoder.W'] = state_dict['encoder.W'][:, :embedding_space_size, :]
    
        model.load_state_dict(old_state_dict, strict=False)
        print("| Successfully loaded initializer model", specific_dict_path)
    #else:
        #print(f"State dictionary file '{state_dict_path}' does not exist. Continuing without loading pre-trained weights.")


    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    # Uncomment below to test a Circuit Length of 0 (no time evolution)
    params = generate_circuit_params(0,12)
    #params = generate_circuit_params(file_name = directory_path + 'dense_small.param')
    dataset = QuantumSimulationDatasetFast(params, batch_size, 4, device, inverse_density=3)

    for i, (final_state, initial_state) in enumerate(dataset):
        
        optimizer.zero_grad()
        loss, loss2 = model.test_objective_function(initial_state, final_state)
        loss = loss.mean()
        loss.backward()
        
        expected_mutual_info = mutual_info_score(initial_state.view(-1), final_state.view(-1))
        initial_entropy = entropy(initial_state)
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Adjust max_norm as needed
        
        # if i % 10 == 0:
        print('| Iteration', i, ' > I(W,Z):', f"{-loss.detach().cpu().item():,.5f}", 
              ' I(W,Z):', f"{loss2:,.5f}",
              ' I(X,Y):', f"{expected_mutual_info:,.5f}",
              ' H(X):', f"{initial_entropy:,.5f}")
        
        optimizer.step()
        if i > num_steps:
            print('Training Terminated')
            break
        if i % 1000 == 999:
            torch.save(model.state_dict(), f'quantum_experiments/experiment_{state_space}_{embedding_space_size}_{i}.model')

    # Save the model at the end of training
    final_save_path = f'quantum_experiments/experiment_{state_space}_{embedding_space_size}_final.model'
    torch.save(model.state_dict(), final_save_path)
    print(f"Final model saved to {final_save_path}")

# Hyperparameter Tuning

In [176]:
def find_learning_rate(device, state_space, embedding_space_size, batch_size=256, num_steps=300):
    model = EmbeddingMI3(batch_size, state_space, embedding_space_size, num_ones=4).to(device)
    
    # Load parameters (adapt as needed)
    params = generate_circuit_params(file_name=directory_path + 'dense_small.param')
    dataset = QuantumSimulationDatasetFast(params, batch_size, 4, device, inverse_density=3)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-7)
    lrs = []
    losses = []
    lr = 1e-7

    for i, (final_state, initial_state) in enumerate(dataset):
        if i > num_steps:
            break
        
        optimizer.zero_grad()
        loss = model.test_objective_function(initial_state, final_state).mean()
        loss.backward()
        optimizer.step()

        lrs.append(lr)
        losses.append(loss.item())

        # Increase learning rate
        lr *= 1.05
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        
        if i % 10 == 0:
            print(f'| Iteration {i}, Learning Rate: {lr:.8f}, Loss: {loss.item():.8f}')
    
    # Plot results
    plt.plot(lrs, losses)
    plt.xscale('log')
    plt.xlabel('Learning Rate')
    plt.ylabel('Loss')
    plt.title('Learning Rate Finder')
    plt.show()

# Main

In [177]:
if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device Running: ', device)

    # experiments = [
    #     (12, 2, 1024), (12, 3, 1024), (12, 4, 1024), (12, 5, 1024),
    #     (12, 6, 1024), (12, 7, 1024), (12, 8, 1024), (12, 9, 1024),
    #     (12, 10, 1024), (12, 11, 1024), (12, 12, 1024)
    # ]
    
    experiments = [
        (12, 12, 1024), (12, 2, 1024), (12, 4, 1024), (12, 8, 1024)
    ]

    for i, params in enumerate(experiments):
        print('Running Experiment', i, params)
        print('State Space of', params[0], 'to Embedding Space of', params[1])
        
        run_dim_red_process(device, *params)
        #find_learning_rate(device, *params)



Device Running:  cpu
Running Experiment 0 (12, 12, 1024)
State Space of 12 to Embedding Space of 12
(Parameter containing:
tensor([[ 0.1279,  0.1062, -0.1326, -0.6593, -0.0706,  0.4276,  0.4161,  0.2417,
          0.2607,  0.5498, -0.0252,  0.3133]], requires_grad=True), Parameter containing:
tensor([[[-0.0373,  0.0663,  0.0657,  0.0833,  0.0306,  0.0435,  0.0764,
          -0.0398,  0.0902,  0.0223, -0.1645,  0.1574],
         [ 0.1946,  0.1469, -0.0794, -0.0907,  0.0555, -0.1393,  0.0500,
           0.1475,  0.0031,  0.0126, -0.0116, -0.1408],
         [ 0.1231,  0.1174, -0.0144, -0.1857, -0.0639,  0.1828,  0.1595,
          -0.0423,  0.1322,  0.0791, -0.0023,  0.1949],
         [-0.1502, -0.0329, -0.1415,  0.0494, -0.0933,  0.0181,  0.1100,
          -0.0430, -0.0688, -0.1138,  0.0610,  0.0402],
         [-0.1170,  0.0720, -0.1014, -0.1869,  0.0835,  0.1954, -0.1362,
           0.1644, -0.0825,  0.0919, -0.0766,  0.1027],
         [-0.1635,  0.1298, -0.1845,  0.0018, -0.1289,  0.099

KeyboardInterrupt: 